# Refactor the Taxi Fare Prediction Problem with a Pipeline

We will refactor the model you built for the Taxi Fare Prediction Problem using:
- Custom encoders for the distance and time features
- OneHot Encoder in order to encode the hour and day of week features
- SimpleImputer to fill missing values
- A simple linear regression
- A pipeline to put all together

Then: 
- train this pipeline
- apply the pipeline on test data
- generate predictions and submit these new predictions to Kaggle

## First pipeline

In [2]:
# import the train dataset ( 1000 first rows)
import pandas as pd

df = pd.read_csv("../data/train.csv", nrows=1000)

In [3]:
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
995,2011-07-30 07:33:13.0000001,16.9,2011-07-30 07:33:13 UTC,-73.884801,40.755707,-73.980472,40.765556,1
996,2011-09-24 23:20:00.000000112,13.7,2011-09-24 23:20:00 UTC,-73.953603,40.779203,-73.995763,40.726701,3
997,2011-12-22 11:07:00.00000037,29.3,2011-12-22 11:07:00 UTC,-73.942380,40.837712,-73.864372,40.769985,2
998,2014-11-03 12:40:00.00000014,6.5,2014-11-03 12:40:00 UTC,-73.961151,40.774578,-73.972251,40.785640,1


In [7]:
# prepare X and y
X = df.drop(columns=["fare_amount"])
y = df.fare_amount

In [9]:
# Hold out ( train and test dplit )
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
X_train.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
541,2014-07-07 13:52:00.000000120,2014-07-07 13:52:00 UTC,-73.979962,40.759952,-73.994670,40.723662,1
440,2010-01-31 15:21:08.0000002,2010-01-31 15:21:08 UTC,-73.976061,40.758180,-73.948436,40.782550,1
482,2013-11-19 10:38:00.00000061,2013-11-19 10:38:00 UTC,-74.003197,40.744075,-74.000670,40.726062,1
422,2010-04-21 13:39:00.000000112,2010-04-21 13:39:00 UTC,-73.988323,40.722913,-74.002315,40.729572,1
778,2011-02-07 08:39:21.0000005,2011-02-07 08:39:21 UTC,-73.964999,40.806719,-73.958495,40.784222,1


### Custom transformers

With the Taxi Fare Prediction Challenge data, using `BaseEstimator` and `TransformerMixin`, implement:

- a transformer that computes the haversine distance between the pickup and dropoff locations
- a custom encoder that extracts the time features from `pickup_datetime`

In [11]:
import numpy as np

def haversine_vectorized(df, 
                         start_lat="pickup_latitude",
                         start_lon="pickup_longitude",
                         end_lat="dropoff_latitude",
                         end_lon="dropoff_longitude"):
    """ 
        Calculates the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df.
        Computes the distance in kms.
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [23]:
# create a DistanceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(BaseEstimator, TransformerMixin):
    """
        Computes the haversine distance between two GPS points.
        Returns a copy of the DataFrame X with only one column: 'distance'.
    """

    def __init__(self,
                 start_lat="pickup_latitude",
                 start_lon="pickup_longitude",
                 end_lat="dropoff_latitude",
                 end_lon="dropoff_longitude"):

        self.start_lat = start_lat
        self.start_lon = start_lon
        self.end_lat = end_lat
        self.end_lon = end_lon
        # A COMPPLETER
    
    def fit(self, X, y=None):
        return self
        # A COMPLETER 

    def transform(self, X, y=None):
        lat_1_rad, lon_1_rad = np.radians(X[self.start_lat].astype(float)), np.radians(X[self.start_lon].astype(float))
        lat_2_rad, lon_2_rad = np.radians(X[self.end_lat].astype(float)), np.radians(X[self.end_lon].astype(float))
        dlon = lon_2_rad - lon_1_rad
        dlat = lat_2_rad - lat_1_rad
        a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
        c = 2 * np.arcsin(np.sqrt(a))
        return pd.DataFrame(6371 * c, columns=["distance"])

In [24]:
# test the DistanceTransformer

dist_trans = DistanceTransformer()
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

,distance
541,4.221228
440,3.571415
482,2.014239
422,1.392289
778,2.560767


In [45]:
# create a TimeFeaturesEncoder
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """
        Extracts the day of week (dow), the hour, the month and the year from a time column.
        Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'.
    """

    def __init__(self, datetime="pickup_datetime", time_zone ="America/New_York"):
        self.datetime = datetime    
        self.time_zone = time_zone
       # A COMPLETER

    def fit(self, X, y=None):
        return self
        # A COMPLETER

    def transform(self, X, y=None):
        X[self.datetime] = pd.to_datetime(X[self.datetime], format="%Y-%m-%d %H:%M:%S %Z").dt.tz_convert(self.time_zone)
        X_ = pd.DataFrame()
        X_["dow"] = X[self.datetime].dt.dayofweek
        X_["hour"] = X[self.datetime].dt.hour
        X_["month"] = X[self.datetime].dt.month
        X_["year"] = X[self.datetime].dt.year

        # A COMPLETER
        return X_

In [46]:
time_enc = TimeFeaturesEncoder('pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

,dow,hour,month,year
541,0,9,7,2014
440,6,10,1,2010
482,1,5,11,2013
422,2,9,4,2010
778,0,3,2,2011


In [11]:
# test the TimeFeaturesEncoder

time_enc = TimeFeaturesEncoder('pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

,dow,hour,month,year
pickup_datetime,,,,
2013-06-14 13:37:00-04:00,4,13,6,2013
2014-06-14 16:48:12-04:00,5,16,6,2014
2015-03-22 10:23:12-04:00,6,10,3,2015
2009-09-13 08:47:43-04:00,6,8,9,2009
2014-07-20 08:48:35-04:00,6,8,7,2014


###  Prepocessing pipeline

In [47]:
# visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')

#### Distance pipeline

Create a pipeline for distances:
- convert the pickup and dropoff coordinates into distances with the DistanceTransformer
- standardize these distances

In [48]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# create distance pipeline dist_pipe
dist_pipe = Pipeline([("dist_trans", DistanceTransformer()), ("std_scale", StandardScaler())])

# display distance pipeline
dist_pipe

Pipeline(steps=[('dist_trans', DistanceTransformer()),
                ('std_scale', StandardScaler())])

#### Time features pipeline

Create a pipeline for time features
- extract time features from pickup datetime with the TimeFeaturesEncoder
- encode these categorical time features with the OneHotEncoder

In [56]:
from sklearn.preprocessing import OneHotEncoder
# create time pipeline time_pipe
time_pipe = Pipeline([("time_encode", TimeFeaturesEncoder()), ("one_hot", OneHotEncoder(handle_unknown="ignore"))])

# display time pipeline
time_pipe

Pipeline(steps=[('time_encode', TimeFeaturesEncoder()),
                ('one_hot', OneHotEncoder(handle_unknown='ignore'))])

#### Preprocessing pipeline

Wrap up the distance pipeline and the time pipeline into a preprocessing pipeline.

In [59]:
from sklearn.compose import ColumnTransformer
# create preprocessing pipeline preproc_pipe
preproc_pipe = ColumnTransformer(transformers=[("distance", dist_pipe, 
                                                    ["pickup_latitude", 
                                                    "pickup_longitude", 
                                                    "dropoff_latitude", 
                                                    "dropoff_longitude"]),
                                                ("time", time_pipe,
                                                    ["pickup_datetime"])])
# display preprocessing pipeline
preproc_pipe

ColumnTransformer(transformers=[('distance',
                                 Pipeline(steps=[('dist_trans',
                                                  DistanceTransformer()),
                                                 ('std_scale',
                                                  StandardScaler())]),
                                 ['pickup_latitude', 'pickup_longitude',
                                  'dropoff_latitude', 'dropoff_longitude']),
                                ('time',
                                 Pipeline(steps=[('time_encode',
                                                  TimeFeaturesEncoder()),
                                                 ('one_hot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pickup_datetime'])])

### Model pipeline

Create a pipeline containing the preprocessing and the regression model of your choice.

In [60]:
from sklearn.linear_model import LinearRegression

# Add the model of your choice to the pipeline nammed pipe
pipe = Pipeline([("preproc", preproc_pipe), ("model", LinearRegression())])
# display the pipeline with model
pipe

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('dist_trans',
                                                                   DistanceTransformer()),
                                                                  ('std_scale',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude']),
                                                 ('time',
                                                  Pipeline(steps=[('time_encode',
                                                                   TimeFeaturesEncoder()),
                                                                  ('one_hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pickup_datetime'])])),
                ('model', LinearRegression())])

### Training and performance

Train the pipelined model and compute the prediction on the test set:

In [64]:
# train the pipelined model

pipe.fit(X_train, y_train)
# compute y_pred on the test set
y_pred = pipe.predict(X_test)

Use the RMSE to evaluate the performance of the model:

In [65]:
def compute_rmse(y_pred, y_true):
    return np.sqrt(((y_pred - y_true)**2).mean())

In [66]:
# call compute_rmse
compute_rmse(y_pred, y_test)

9.653808294503953

## Complete the workflow with a pipeline

Here we will implement the whole workflow for our Taxifare kaggle challenge.

For that we will refactor the code in functions for more clarity.

Implement the following functions:
- `get_data()` to fetch the data 
- `clean_data()` to clean the data
- `get_pipeline()` to get the pipeline defined earlier
- `train()` to train our model
- `evaluate()` to evaluate our model on test data

In [91]:
# implement get_data() function
def get_data(nrows=1000000):
    '''returns a DataFrame with nrows '''
    df = pd.read_csv("../data/train.csv", nrows=nrows)
    return df

In [73]:
# implement clean_data() function
def clean_data(df, test=False):
    '''returns a DataFrame without outliers and missing values'''
    # A COMPLETER
    return df[(df.fare_amount.between(0, 60)) & 
            (df.passenger_count < 9 ) & 
            (df.passenger_count != 0) & 
            (df.pickup_latitude.between(left = 40, right = 42 )) & 
            (df.pickup_longitude.between(left = -74.3, right = -72.9 )) & 
            (df.dropoff_latitude.between(left = 40, right = 42 )) & 
            (df.dropoff_longitude.between(left = -74, right = -72.9 ))]

In [70]:
# implement set_pipeline() function
def set_pipeline():
    '''returns a pipelined model'''
    dist_pipe = Pipeline([("dist_trans", DistanceTransformer()), ("std_scale", StandardScaler())])
    time_pipe = Pipeline([("time_encode", TimeFeaturesEncoder()), ("one_hot", OneHotEncoder(handle_unknown="ignore"))])
    preproc_pipe = ColumnTransformer(transformers=[("distance", dist_pipe, 
                                                    ["pickup_latitude", 
                                                    "pickup_longitude", 
                                                    "dropoff_latitude", 
                                                    "dropoff_longitude"]),
                                                ("time", time_pipe,
                                                    ["pickup_datetime"])])
    pipe = Pipeline([("preproc", preproc_pipe), ("model", LinearRegression())])
     # A  COMPLETER
    return pipe

In [75]:
# implement train() function
def train(X_train, y_train, pipeline):
    '''returns a trained pipelined model'''
    pipeline.fit(X_train, y_train)
   # A COMPLETER
    return pipeline

In [71]:
# implement evaluate() function
def evaluate(X_test, y_test, pipeline):
    '''returns the value of the RMSE'''
    y_pred = pipeline.predict(X_test)
    # A COMPLETER
    return np.sqrt(((y_pred - y_test)**2).mean())

### Test the complete worflow

Use the above functions to test the complete workflow.

In [93]:
# store the data in a DataFrame
df = get_data()
df = clean_data(df)
# set X and y
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
pipeline = train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)
print(rmse)

4.4741449883573905


### Congrats!

Now we are ready to convert this complete workflow into a packaged code 🚀